# Using the Fixed Variance process
The `FixedVariance` volatility process can be used to implement zig-zag model 
estimation where two steps are repeated until convergence.  This can be used 
to estimate models which may not be easy to estimate as a single process due
to numerical issues or a high-dimensional parameter space.

_This setup code is required to run in an IPython notebook_

In [ ]:
import matplotlib.pyplot as plt
import seaborn

seaborn.set_style("darkgrid")
plt.rc("figure", figsize=(16, 6))
plt.rc("savefig", dpi=90)
plt.rc("font", family="sans-serif")
plt.rc("font", size=14)

### Setup

Imports used in this example.

In [ ]:
import numpy as np

## Data
The VIX index will be used to illustrate the use of the `FixedVariance` process.  The data is from FRED and is provided by the `arch` package.

In [ ]:
import arch.data.vix

vix_data = arch.data.vix.load()
vix = vix_data.vix.dropna()
vix.name = "VIX Index"
ax = vix.plot(title="VIX Index")

## Initial Mean Model Estimation
The first step is to estimate the mean to filter the residuals using a constant variance. 

In [ ]:
from arch.univariate.mean import HARX, ZeroMean
from arch.univariate.volatility import GARCH, FixedVariance

mod = HARX(vix, lags=[1, 5, 22])
res = mod.fit()
print(res.summary())

## Initial Volatility Model Estimation
Using the previously estimated residuals, a volatility model can be estimated using a `ZeroMean`. In this example, a GJR-GARCH process is used for the variance.

In [ ]:
vol_mod = ZeroMean(res.resid.dropna(), volatility=GARCH(p=1, o=1, q=1))
vol_res = vol_mod.fit(disp="off")
print(vol_res.summary())

In [ ]:
ax = vol_res.plot("D")

## Re-estimating the mean with a ``FixedVariance``
The `FixedVariance` requires that the variance is provided when initializing the object.  The variance provided should have the same shape as the original data.  Since the variance estimated from the GJR-GARCH model is missing the first 22 observations due to the HAR lags, we simply fill these with 1.  These values will not be used to estimate the model, and so the value is not important. 

The summary shows that there is a single parameter, ``scale``, which is close to 1. The mean parameters
have changed which reflects the GLS-like weighting that this re-estimation imposes.

In [ ]:
variance = np.empty_like(vix)
variance.fill(1.0)
variance[22:] = vol_res.conditional_volatility**2.0
fv = FixedVariance(variance)
mod = HARX(vix, lags=[1, 5, 22], volatility=fv)
res = mod.fit()
print(res.summary())

## Zig-Zag estimation
A small repetitions of the previous two steps can be used to implement a so-called zig-zag estimation strategy.

In [ ]:
for i in range(5):
    print(i)
    vol_mod = ZeroMean(res.resid.dropna(), volatility=GARCH(p=1, o=1, q=1))
    vol_res = vol_mod.fit(disp="off")
    variance[22:] = vol_res.conditional_volatility**2.0
    fv = FixedVariance(variance, unit_scale=True)
    mod = HARX(vix, lags=[1, 5, 22], volatility=fv)
    res = mod.fit(disp="off")
print(res.summary())

## Direct Estimation
This model can be directly estimated.  The results are provided for comparison to the previous 
``FixedVariance`` estimates of the mean parameters.

In [ ]:
mod = HARX(vix, lags=[1, 5, 22], volatility=GARCH(1, 1, 1))
res = mod.fit(disp="off")
print(res.summary())